# Pandas Methods & Indexing — Case Lab (Fundamentals + Insight)

Bu notebook, **Pandas fundamentals** kapsamında şu becerileri tek bir mini-case üzerinde pekiştirir:

- **Inspection**: veriyi hızlı tanıma (`head`, `shape`, `info`)
- **Attributes vs Methods** ayrımı (parantez farkı)
- Temel **methods**: `describe`, `mean`, `value_counts`, `isna`
- **Indexing/Selection**: `.loc` ve `.iloc`
- Küçük ama anlamlı **insight** üretimi (groupby ile bağlam kurma)

> Dataset: `Lending-company.csv` (repo içindeki `data/` klasöründen okunur)


In [1]:
import pandas as pd

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)


In [2]:
# Raw Data Protection Pattern:
# df_raw -> dokunulmaz ham veri
# df     -> üzerinde çalıştığımız kopya

df_raw = pd.read_csv("../data/Lending-company.csv")
df = df_raw.copy()

df.head()


,LoanID,StringID,Product,CustomerGender,Location,Region,TotalPrice,StartDate,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360,LoanStatus
0,1,LoanID_1,Product B,Female,Location 3,Region 2,17600.0,04/07/2018,2200,45,365,3221,4166,14621,Active
1,2,LoanID_2,Product D,Female,Location 6,Region 6,NaN,02/01/2019,2200,45,365,3161,4096,16041,Active
2,3,LoanID_3,Product B,Male,Location 8,Region 3,16600.0,08/12/2016,1000,45,365,2260,3205,16340,NaN
3,4,LoanID_4,Product A,Male,Location 26,Region 2,17600.0,NaN,2200,45,365,3141,4166,16321,Active
4,5,LoanID_5,Product B,Female,Location 34,Region 3,21250.0,28/10/2017,2200,55,365,3570,4745,14720,Active


## 1) Quick Inspection (İlk Bakış)

Bu bölümün amacı: veriyle “tanışmak”.


In [3]:
df.shape, df.columns

((1043, 15),
 Index(['LoanID', 'StringID', 'Product', 'CustomerGender', 'Location', 'Region', 'TotalPrice', 'StartDate', 'Deposit',
        'DailyRate', 'TotalDaysYr', 'AmtPaid36', 'AmtPaid60', 'AmtPaid360', 'LoanStatus'],
       dtype='object'))

In [4]:
# info(): kolonlar, dtype'lar, eksik değerler (non-null), memory kullanımı
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1043 entries, 0 to 1042
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   LoanID          1043 non-null   int64  
 1   StringID        1043 non-null   object 
 2   Product         1043 non-null   object 
 3   CustomerGender  1043 non-null   object 
 4   Location        1043 non-null   object 
 5   Region          1042 non-null   object 
 6   TotalPrice      1018 non-null   float64
 7   StartDate       1042 non-null   object 
 8   Deposit         1043 non-null   int64  
 9   DailyRate       1043 non-null   int64  
 10  TotalDaysYr     1043 non-null   int64  
 11  AmtPaid36       1043 non-null   int64  
 12  AmtPaid60       1043 non-null   int64  
 13  AmtPaid360      1043 non-null   int64  
 14  LoanStatus      1006 non-null   object 
dtypes: float64(1), int64(7), object(7)
memory usage: 122.4+ KB


## 2) Attributes vs Methods

- **Attributes**: parantez yok → hızlı bilgi (örn. `df.shape`)
- **Methods**: parantez var → işlem/hesap (örn. `df.describe()`)

> Kural: `()` görüyorsan bir iş yaptırıyorsun.


In [5]:
# Attribute örnekleri
df.shape


(1043, 15)

In [6]:
# Method örnekleri
df.describe()


,LoanID,TotalPrice,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360
count,1043.000000,1018.000000,1043.000000,1043.000000,1043.0,1043.000000,1043.000000,1043.000000
mean,522.000000,19562.843811,2528.092042,50.834132,365.0,3901.857143,5214.393097,17122.131352
std,301.232468,5955.588065,1504.467724,11.567390,0.0,2537.597667,2850.327133,5910.916736
min,1.000000,13475.000000,1000.000000,30.000000,365.0,-2770.000000,-2945.000000,650.000000
25%,261.500000,16300.000000,2200.000000,45.000000,365.0,2840.500000,3822.000000,15560.500000
50%,522.000000,17600.000000,2200.000000,45.000000,365.0,3420.000000,4742.000000,16617.000000
75%,782.500000,20950.000000,2200.000000,55.000000,365.0,5075.500000,6393.000000,19977.500000
max,1043.000000,70225.000000,8000.000000,150.000000,365.0,18851.000000,22143.000000,65001.000000


## .describe() Çıktısı Üzerine Veri Kalitesi ve Analizi

`.describe()` fonksiyonu bize sayısal değişkenler için temel istatistikleri verir:

- **count** → Non-null gözlem sayısı  
- **mean** → Ortalama  
- **std** → Standart sapma  
- **min / max** → Aralık  
- **25%, 50%, 75%** → Çeyrekler (quantile)

---

### Eksik Veri Tespiti

- Veri setinde toplam **1043 gözlem** bulunmaktadır.
- `TotalPrice` kolonunda **1018 gözlem** vardır.
- Bu durum ilgili değişkende **25 adet eksik değer (NaN)** olduğunu göstermektedir.
 Bu durum veri kalitesi açısından not edilmelidir ve analiz öncesinde eksik değer stratejisi belirlenmelidir.

---

### Negatif Ödeme Değerleri (Kritik Gözlem)

Aşağıdaki değişkenlerde negatif minimum değerler gözlenmiştir:

- `AmtPaid36` → min = -2770  
- `AmtPaid60` → min = -2945  

 Finansal bir veri setinde negatif ödeme değerleri:

- İade (refund)
- Chargeback
- Veri giriş hatası
- Muhasebe düzeltme kaydı

gibi durumları işaret edebilir.

Bu değerler analiz öncesinde doğrulanmalı ve anlamı netleştirilmelidir.

---

### Sabit Kolon Problemi

`TotalDaysYr` değişkeni için:

- mean = 365  
- std = 0  
- min = 365  
- max = 365  

Bu durum tüm gözlemlerin 365 olduğunu göstermektedir.

  Bu değişken varyasyon içermediği için:
- Modelleme açısından bilgi taşımaz.
- Analitik katkısı düşüktür.
- Feature olarak çıkarılması değerlendirilebilir.

---

### Deposit Değişkeni Dağılımı

`Deposit` değişkeninde:

- 25%, 50% ve 75% değerlerinin 2200 olması,
- Ödemelerin büyük ölçüde bu değerde yoğunlaştığını göstermektedir.

 Bu durum değişkenin ayrık (discrete) ve kümelenmiş bir dağılıma sahip olduğunu düşündürmektedir.

---

### Genel Analitik Değerlendirme

- Veri seti genel olarak tutarlı görünmektedir.
- Eksik veri sınırlıdır (yalnızca `TotalPrice` değişkeninde).
- Negatif ödeme değerleri dikkatle incelenmelidir.
- Sabit kolonlar analiz gücünü düşürebilir.
- Deposit dağılımı homojen değildir.

Bu gözlemler veri temizleme ve ileri analiz aşamalarında dikkate alınmalıdır.

## 3) Working with Methods (Sayısal + Kategorik)

Önce sayısal kolonları ayırt edelim; sonra uygun method’ları seçelim.


In [7]:
numeric_cols = df.select_dtypes(include="number").columns
categorical_cols = df.select_dtypes(exclude="number").columns

numeric_cols, categorical_cols


(Index(['LoanID', 'TotalPrice', 'Deposit', 'DailyRate', 'TotalDaysYr', 'AmtPaid36', 'AmtPaid60', 'AmtPaid360'], dtype='object'),
 Index(['StringID', 'Product', 'CustomerGender', 'Location', 'Region', 'StartDate', 'LoanStatus'], dtype='object'))

In [8]:
# Sayısal özet (describe)
df[numeric_cols].describe()


,LoanID,TotalPrice,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360
count,1043.000000,1018.000000,1043.000000,1043.000000,1043.0,1043.000000,1043.000000,1043.000000
mean,522.000000,19562.843811,2528.092042,50.834132,365.0,3901.857143,5214.393097,17122.131352
std,301.232468,5955.588065,1504.467724,11.567390,0.0,2537.597667,2850.327133,5910.916736
min,1.000000,13475.000000,1000.000000,30.000000,365.0,-2770.000000,-2945.000000,650.000000
25%,261.500000,16300.000000,2200.000000,45.000000,365.0,2840.500000,3822.000000,15560.500000
50%,522.000000,17600.000000,2200.000000,45.000000,365.0,3420.000000,4742.000000,16617.000000
75%,782.500000,20950.000000,2200.000000,55.000000,365.0,5075.500000,6393.000000,19977.500000
max,1043.000000,70225.000000,8000.000000,150.000000,365.0,18851.000000,22143.000000,65001.000000


In [9]:
# Parametre/argument örneği:
# numeric_only=True -> sadece sayısal kolonlarda ortalama al
df.mean(numeric_only=True)


LoanID           522.000000
TotalPrice     19562.843811
Deposit         2528.092042
DailyRate         50.834132
TotalDaysYr      365.000000
AmtPaid36       3901.857143
AmtPaid60       5214.393097
AmtPaid360     17122.131352
dtype: float64

### Kategorik Özet: value_counts()

Kategorik kolonlarda ortalama yerine genellikle dağılım (frekans) bakılır.


In [10]:
df["LoanStatus"].value_counts(dropna=False)


LoanStatus
Finished Payment    534
Active              460
NaN                  37
Blocked              12
Name: count, dtype: int64

## 4) Data Quality Check (Missing Values)

Eksik değer nerede var? Bu aşamada sadece **tespit** ediyoruz (temizleme sonraki notebook'ta).


In [11]:
df.isna().sum().sort_values(ascending=False)


LoanStatus        37
TotalPrice        25
Region             1
StartDate          1
LoanID             0
StringID           0
Product            0
CustomerGender     0
Location           0
Deposit            0
DailyRate          0
TotalDaysYr        0
AmtPaid36          0
AmtPaid60          0
AmtPaid360         0
dtype: int64

## 5) From Summary to Selection (loc / iloc)

Genel özet iyi bir başlangıçtır. Analizde güç, belirli grupları seçip inceleyebilmekten gelir.

Kalıp:
- `df.loc[koşul, kolonlar]` → label-based seçim
- `df.iloc[satır_indexi, kolon_indexi]` → position-based seçim


### Scenario: Region 2 ve yüksek TotalPrice

Soru: Region 2'de **ortalama TotalPrice** nedir?


In [12]:
df.loc[df["Region"] == "Region 2", "TotalPrice"].mean()


19308.016304347828

Soru: TotalPrice > 20000 olan kayıtları görelim (ilk 5 satır).

In [13]:
high_value_loans = df.loc[df["TotalPrice"] > 20000]
high_value_loans.head()


,LoanID,StringID,Product,CustomerGender,Location,Region,TotalPrice,StartDate,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360,LoanStatus
4,5,LoanID_5,Product B,Female,Location 34,Region 3,21250.0,28/10/2017,2200,55,365,3570,4745,14720,Active
6,7,LoanID_7,Product A,Male,Location 25,NaN,21250.0,04/07/2020,2200,55,365,1951,3176,18701,Active
8,9,LoanID_9,Product A,Male,Location 156,Region 6,23250.0,03/09/2019,5000,55,365,5850,7375,21250,NaN
9,10,LoanID_10,Product C,Male,Location 21,Region 9,21250.0,25/07/2020,2200,55,365,2051,3176,18351,Active
11,12,LoanID_12,Product D,Female,Location 25,Region 6,21250.0,29/03/2017,2200,55,365,3190,4115,11790,Active


Soru: Region 2 **ve** TotalPrice > 20000 olanları çekelim.

In [14]:
df.loc[(df["Region"] == "Region 2") & (df["TotalPrice"] > 20000)].head()


,LoanID,StringID,Product,CustomerGender,Location,Region,TotalPrice,StartDate,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360,LoanStatus
25,26,LoanID_26,Product A,Male,Location 75,Region 2,23250.0,11/09/2019,5000,55,365,6400,6525,15800,Active
42,43,LoanID_43,Product A,Female,Location 42,Region 2,20250.0,17/10/2019,1000,55,365,-200,125,9918,Active
75,76,LoanID_76,Product A,Male,Location 11,Region 2,23250.0,11/09/2019,5000,55,365,5450,6725,18400,Active
76,77,LoanID_77,Product A,Male,Location 75,Region 2,20250.0,31/01/2020,1000,55,365,900,1775,13650,Active
80,81,LoanID_81,Product A,NotSpecified,Location 44,Region 2,20250.0,28/11/2019,1000,55,365,500,1175,9450,Active


### iloc: Konumsal seçim

Bazen “ilk satır”, “ilk 5 satırın ilk 3 kolonu” gibi konum bazlı seçim yapmak isteriz.


In [15]:
df.iloc[0]


LoanID                     1
StringID            LoanID_1
Product            Product B
CustomerGender        Female
Location          Location 3
Region              Region 2
TotalPrice           17600.0
StartDate         04/07/2018
Deposit                 2200
DailyRate                 45
TotalDaysYr              365
AmtPaid36               3221
AmtPaid60               4166
AmtPaid360             14621
LoanStatus            Active
Name: 0, dtype: object

In [16]:
df.iloc[0:5, 0:3]


,LoanID,StringID,Product
0,1,LoanID_1,Product B
1,2,LoanID_2,Product D
2,3,LoanID_3,Product B
3,4,LoanID_4,Product A
4,5,LoanID_5,Product B


## 6) Mini Insight: Region Bazında Bağlam

Tek bir region'a bakmak yerine tüm region'ları kıyaslayalım:
- Ortalama TotalPrice (mean)
- Kaç kayıt var? (count)

Bu, “ortalama tek başına yetmez” refleksini öğretir.


In [17]:
region_summary = (
    df.groupby("Region")
      .agg(
          avg_total_price=("TotalPrice", "mean"),
          loan_count=("LoanID", "count")
      )
      .sort_values(by="avg_total_price", ascending=False)
)

region_summary


,avg_total_price,loan_count
Region,,
Region 3,20345.625000,206
Region 14,19950.000000,3
Region 4,19872.177419,62
Region 1,19770.200000,131
Region 10,19707.142857,7
Region 15,19525.000000,8
Region 6,19439.375000,326
Region 17,19400.000000,3
Region 2,19308.016304,188


### Insight Interpretation (kısa)

- Ortalama yüksek olan region'lar her zaman “en önemli” değildir; **kaç gözleme** dayandığı önemlidir.
- Bu tabloyu okurken iki şeyi birlikte düşün:
  1) `avg_total_price` (seviye)
  2) `loan_count` (güvenilirlik/temsil)


## Methods & Indexing Lab Summary

Bu notebook’ta Pandas temel metodları ve indeksleme mantığı uygulamalı olarak çalışılmıştır.

### 1. DataFrame Yapısı ve Kolon Seçimi
- Tekli ve çoklu kolon seçimi uygulanmıştır.
- Label-based (`loc`) ve position-based (`iloc`) indexing kullanılmıştır.
- Boolean filtering ile koşullu veri seçimi yapılmıştır.

### 2. Veri Filtreleme ve Koşullu Seçim
- Sayısal eşiklere göre filtreleme
- Kategorik değerlere göre filtreleme
- Birden fazla koşul ile kombinasyonlu filtreleme

Bu bölüm veri alt kümeleri ile çalışma pratiği kazandırmıştır.

### 3. Metod Zincirleme (Method Chaining)
- `sort_values`
- `value_counts`
- `unique`
- `nunique`
- `head` / `tail`

Bu metodlar analitik keşif sürecinin temel yapı taşlarıdır.

### 4. GroupBy Mantığına Giriş
- `groupby` ile kategorik bazlı özet istatistikler
- Ortalama ve sayım hesaplama
- Sıralama ile karşılaştırma

### 5. Analitik Kazanım

Bu notebook sonunda:

✔ Veri alt kümeleri oluşturabilme  
✔ Koşullu filtreleme yapabilme  
✔ Kategorik bazlı özet çıkarabilme  
✔ Veri yapısını okuma becerisi  
✔ Pandas indexing mantığını anlama  

yetkinlikleri kazanılmıştır.

Bu çalışma modelleme değil, veri düşünme pratiğidir.